In [1]:
## this is the code required portions of the PSET // later to be transferred to
## to the LaTeX homework sheet

In [4]:
## IMPORTS
import numpy as np
import numpy.random as rn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from math import log, factorial
from scipy.optimize import minimize

In [12]:
def neg_log_likelihood(lambda_, obs):
    ## direct implementation of the equation found in q1 
    sum_x_i = sum(obs)
    log_fact_sum = sum(log(factorial(x)) for x in obs)
    neg_log_likelihood = 8 * lambda_ - sum_x_i * log(lambda_) + log_fact_sum
    return neg_log_likelihood

n_estimate = neg_log_likelihood(
    20, [22, 23, 23, 12, 18, 25, 23, 14])
print(n_estimate)


23.551530395903967
